In [3]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [11]:
legal_text = pd.read_excel('data/clean_statutes.xlsx')
legal_text.head()

,Section,Subsection,Section_Title,Section_Text
0,151,1.0,SHORT TITLE,"This chapter may be cited as the Limited Sales, Excise, and Use Tax Act."
1,151,2.0,"APPLICABILITY OF DEFINITIONS, ETC","The definitions and other provisions of this chapter relating to the collection, administration, and enforcement of the taxes imposed by this chapter, including the requirements for sales tax permits, apply to the parties to a sale of a taxable item that is exempted from the taxes imposed by this chapter but that is subject to the taxes imposed by a city under Chapter 321 of this code."
2,151,28.0,AMUSEMENT SERVICES,"(a) Amusement services means the provision of amusement, entertainment, or recreation, but does not include the provision of educational or health services if prescribed by a licensed practitioner of the healing arts for the primary purpose of education or health maintenance or improvement.(b) Amusement services includes membership in a private club or organization that provides entertainment, recreational, sports, dining, or social facilities to its members.(c) Amusement services does not include services provided through coin-operated machines that are operated by the consumer."
3,151,3.0,BUSINESS,"Business means an activity of or caused by a person for the purpose of a direct or indirect gain, benefit, or advantage."
4,151,31.0,COMPUTER PROGRAM,"Computer program means a series of instructions that are coded for acceptance or use by a computer system and that are designed to permit the computer system to process data and provide results and information. The series of instructions may be contained in or on magnetic tapes, punched cards, printed instructions, or other tangible or electronic media."


In [14]:
model = 'distilbert-base-nli-stsb-mean-tokens'
text_col = 'Section_Text'
group_col = 'Section_Title'

embedder = SentenceTransformer(model)
legal_text['sentence'] = legal_text[text_col].str.split(".",expand=False).values
legal_text = legal_text.explode('sentence')
legal_text = legal_text.loc[legal_text['sentence']!='']
legal_text['sentence_embedding'] = embedder.encode(legal_text.sentence).tolist()
legal_text.head()

,Section,Subsection,Section_Title,Section_Text,sentence,sentence_embedding
0,151,1.0,SHORT TITLE,"This chapter may be cited as the Limited Sales, Excise, and Use Tax Act.","This chapter may be cited as the Limited Sales, Excise, and Use Tax Act","[0.16894295811653137, -0.00851594191044569, -0.296162873506546, 0.2573283612728119, -0.4143689274787903, -0.2717636227607727, 0.7010982632637024, -0.3780181407928467, -0.6480791568756104, 0.7720542550086975, -0.5060065984725952, 0.16790945827960968, -0.13197669386863708, -0.6182047128677368, -0.33469220995903015, 0.0659010037779808, -0.28614526987075806, 0.7763422727584839, -0.17924439907073975, -0.9127068519592285, 0.5247442722320557, -0.2832961976528168, 0.09317886829376221, 0.22836032509803772, 0.31795448064804077, -0.08414573222398758, 0.8347461223602295, 0.14836518466472626, -0.22460508346557617, -1.002913475036621, -0.33680590987205505, 0.3687874376773834, 0.3402027487754822, -0.07263676822185516, 0.4310338497161865, -0.7119324803352356, -0.06199664995074272, -0.8657553791999817, -0.3929443359375, -0.4287366271018982, 0.21567440032958984, -0.5469999313354492, 0.0720679759979248, 0.21653981506824493, -0.20912611484527588, 0.5977379679679871, -0.4017750322818756, 0.25816810131073, -1.0036046504974365, -0.5301874876022339, -0.2023560255765915, -0.2898483872413635, 0.4242514967918396, -0.9273300170898438, -0.400728315114975, -0.1888270378112793, -0.4356876313686371, -0.15422487258911133, 0.40744104981422424, -0.3754659593105316, -0.6607218980789185, 0.17205217480659485, -0.47895827889442444, -0.6150723099708557, -0.07213689386844635, -1.0801880359649658, 0.5136939883232117, 0.7722920179367065, -0.6013466715812683, -1.5178732872009277, -0.06552262604236603, -0.7815015912055969, -1.2493637800216675, 0.3824715316295624, 0.24552591145038605, -0.08070050179958344, 0.21149209141731262, 0.1707715094089508, -0.35965290665626526, -0.32550352811813354, -0.18411791324615479, 0.032342810183763504, -0.46833887696266174, 0.2520596981048584, 0.06257808208465576, -0.7050092220306396, -0.17139853537082672, 0.12051791697740555, 0.17716078460216522, -0.058587558567523956, 0.558040201663971, -0.5767393708229065, 0.3851204514503479, -0.5875386595726013, 0.2765406370162964, -0.5490323901176453, 0.30334892868995667, -0.15269577503204346, -0.3009059429168701, 0.34295323491096497, ...]"
1,151,2.0,"APPLICABILITY OF DEFINITIONS, ETC","The definitions and other provisions of this chapter relating to the collection, administration, and enforcement of the taxes imposed by this chapter, including the requirements for sales tax permits, apply to the parties to a sale of a taxable item that is exempted from the taxes imposed by this chapter but that is subject to the taxes imposed by a city under Chapter 321 of this code.","The definitions and other provisions of this chapter relating to the collection, administration, and enforcement of the taxes imposed by this chapter, including the requirements for sales tax permits, apply to the parties to a sale of a taxable item that is exempted from the taxes imposed by this chapter but that is subject to the taxes imposed by a city under Chapter 321 of this code","[1.055303692817688, -0.07879878580570221, 0.011990068480372429, 0.31049397587776184, -0.06889557838439941, -0.20171038806438446, 0.8443438410758972, -0.7071282863616943, -0.6012312173843384, 0.20297227799892426, -0.812744140625, 0.7422714233398438, -0.9264224171638489, -0.7922874689102173, -0.5745334625244141, -0.6535205245018005, -0.07272425293922424, 0.3950243592262268, -0.29133692383766174, -0.4885833263397217, -0.19814734160900116, -0.23807582259178162, 0.04715336486697197, 0.7785142064094543, 0.5257843136787415, -0.0587129220366478, 0.7513095140457153, 0.23749463260173798, 0.24604998528957367, -0.561417818069458, 0.4457608163356781, 0.27809903025627136, -0.23309840261936188, -0.5301850438117981, -0.053125474601984024, 0.14595060050487518, 0.46928125619888306, -0.6369664072990417, -0.9699912667274475, -0.3232

In [37]:
query = 'natural gas purchase'

pd.set_option('max_colwidth', None)
legal_corpus = legal_text.copy()
query_embedding = embedder.encode(query)
legal_corpus['query_similarity'] = np.round(cosine_similarity(query_embedding, list(legal_corpus.sentence_embedding))[0],3)
legal_corpus=legal_corpus.sort_values('query_similarity', ascending=False).drop_duplicates([group_col])
# legal_corpus[['regulation','service_type','sentence','query_similarity']].head()
legal_corpus[['Section_Title','sentence','query_similarity']].head()

,Section_Title,sentence,query_similarity
117,GAS AND ELECTRICITY,"For purposes of this subsection, the use of gas or electricity in an exempt manner by an independent contractor engaged by the purchaser of the gas or electricity to perform one or more of the exempt activities identified in Subsections (a)(2)-(9) is considered use by the purchaser of the gas or electricity",0.582
102,ITEMS TAXED BY OTHER LAW,(b) Natural gas is exempted under Subsection (a)(2) only to the extent that the gas is taxed as a motor fuel under Chapter 162,0.537
18,SALE FOR RESALE,"(e) A sale for resale does not include the sale of tangible personal property to a purchaser who acquires the property for the purpose of using, consuming, or expending it in, or incorporating it into, an oil or gas well in the performance of an oil well service taxable under Chapter 191",0.485
140,ENERGY-EFFICIENT PRODUCTS,"(a) In this section, energy-efficient product means a product that has been designated as an Energy Star qualified product under the Energy Star program jointly operated by the United States Environmental Protection Agency and the United States Department of Energy",0.463
142,COMPONENTS OF TANGIBLE PERSONAL PROPERTY USED IN CONNECTION WITH SEQUESTRATION OF CARBON DIOXIDE,"003, Health and Safety Code, or a clean energy project, as defined by Section 120",0.446
